In [1]:
pip install ccxt

     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
      --------------------------------------- 0.1/5.7 MB 1.7 MB/s eta 0:00:04
     - -------------------------------------- 0.3/5.7 MB 2.6 MB/s eta 0:00:03
     ---- ----------------------------------- 0.7/5.7 MB 4.9 MB/s eta 0:00:02
     -------- ------------------------------- 1.2/5.7 MB 6.5 MB/s eta 0:00:01
     ---------- ----------------------------- 1.5/5.7 MB 7.1 MB/s eta 0:00:01
     ------------ --------------------------- 1.8/5.7 MB 6.4 MB/s eta 0:00:01
     ------------------- -------------------- 2.8/5.7 MB 8.5 MB/s eta 0:00:01
     ----------------------- ---------------- 3.3/5.7 MB 8.9 MB/s eta 0:00:01
     --------------------------- ------------ 3.9/5.7 MB 9.2 MB/s eta 0:00:01
     ------------------------------- -------- 4.5/5.7 MB 9.5 MB/s eta 0:00:01
     ---------------------------------- ----- 5.0/5.7 MB 9.7 MB/s eta 0:00:01
     -------------------------------------- - 5.6/5.7 MB 9.9 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\xndrf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import ccxt

exchange = ccxt.binance({
    'enableRateLimit': True,  # Важно для соблюдения лимитов запросов
    # 'apiKey': 'YOUR_KEY',   # Опционально, если нужен доступ к закрытым данным
    # 'secret': 'YOUR_SECRET',
})

In [3]:
import pandas as pd
from datetime import datetime

symbol = 'BTC/USDT'
timeframe = '1d'  # Доступные таймфреймы: '1m', '1h', '1d' и т.д.
since = exchange.parse8601('2023-01-01T00:00:00Z')  # Начальная дата

all_ohlcv = []
while True:
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
    if not ohlcv:
        break
    since = ohlcv[-1][0] + 1  # Следующая итерация с нового времени
    all_ohlcv += ohlcv
    print(f"Получено {len(ohlcv)} свечей, всего {len(all_ohlcv)}")
    if len(ohlcv) < 500:  # Большинство бирж возвращают до 1000 свечей за раз
        break

# Конвертация в DataFrame
df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

Получено 500 свечей, всего 500
Получено 376 свечей, всего 876


In [5]:
tickers = exchange.fetch_tickers()
btc_usdt = tickers['BTC/USDT']
print(f"Последняя цена BTC/USDT: {btc_usdt['last']}")

Последняя цена BTC/USDT: 107580.88


In [6]:
df

,open,high,low,close,volume
timestamp,,,,,
2023-01-01,16541.77,16628.00,16499.01,16616.75,96925.413740
2023-01-02,16617.17,16799.23,16548.70,16672.87,121888.571910
2023-01-03,16672.78,16778.40,16605.28,16675.18,159541.537330
2023-01-04,16675.65,16991.87,16652.66,16850.36,220362.188620
2023-01-05,16850.36,16879.82,16753.00,16831.85,163473.566410
...,...,...,...,...,...
2025-05-21,106850.00,110797.38,106100.01,109643.99,45531.040345
2025-05-22,109643.99,111980.00,109177.37,111696.21,31630.773130
2025-05-23,111696.22,111800.00,106800.00,107318.30,31737.723090


In [7]:
pip install pandas_ta

     ---------------------------------------- 0.0/115.1 kB ? eta -:--:--
     ------------------------------- ------- 92.2/115.1 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 115.1/115.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=443e13d51f25c57c52edc769bba71d0dccc2863e55ee1911a422863fd3c7b2a2
  Stored in directory: c:\users\xndrf\appdata\local\pip\cache\wheels\69\00\ac\f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\xndrf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import ccxt
import pandas as pd
import pandas_ta as ta
import time
from datetime import datetime

# 1. Получение исторических данных (с увеличенным лимитом)
def fetch_ohlcv(exchange_name, symbol, timeframe, since, limit=1500):
    try:
        exchange = getattr(ccxt, exchange_name)({
            'enableRateLimit': True,
            'options': {'defaultType': 'future'}
        })
        
        data = []
        since = int(pd.Timestamp(since).timestamp() * 1000)
        
        for _ in range(5):  # Максимум 5 запросов
            try:
                ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
                if len(ohlcv) < 2:
                    break
                data += ohlcv
                since = ohlcv[-1][0] + 1
                time.sleep(exchange.rateLimit / 1000)
            except Exception as e:
                print(f"Ошибка запроса: {e}")
                break
        
        if not data:
            return pd.DataFrame()
            
        df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df.set_index('timestamp').dropna().astype(float)
        
    except Exception as e:
        print(f"Критическая ошибка: {e}")
        return pd.DataFrame()

# 2. Расчет индикаторов (без TA-Lib)
def add_indicators(df):
    if df.empty or len(df) < 100:
        print(f"Недостаточно данных: {len(df)} строк")
        return df
    
    try:
        # Трендовые
        df['SMA_20'] = ta.sma(df['close'], length=20)
        df['EMA_50'] = ta.ema(df['close'], length=50)
        
        # Осцилляторы
        df['RSI_14'] = ta.rsi(df['close'], length=14)
        macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
        df = pd.concat([df, macd], axis=1)
        
        # Объемные
        df['OBV'] = ta.obv(df['close'], df['volume'])
        
        # Волатильность
        bbands = ta.bbands(df['close'], length=20, std=2)
        df = pd.concat([df, bbands], axis=1)
        
        # Свечные паттерны (без TA-Lib)
        df['CDL_HAMMER'] = ta.cdl_hammer(df['open'], df['high'], df['low'], df['close'])
        df['CDL_SHOOTING_STAR'] = ta.cdl_shootingstar(df['open'], df['high'], df['low'], df['close'])
        
        return df.dropna().round(2)
    
    except Exception as e:
        print(f"Ошибка расчета: {e}")
        return df

# 3. Пример использования
if __name__ == "__main__":
    try:
        exchange = "binance"
        symbol = "BTC/USDT"
        timeframe = "1d"
        start_date = "2023-01-01"  # Более свежая дата
        
        print(f"Загрузка данных с {exchange}...")
        df = fetch_ohlcv(exchange, symbol, timeframe, start_date)
        
        if not df.empty:
            print(f"Успешно загружено {len(df)} свечей")
            print("Пример данных до индикаторов:")
            print(df[['close', 'volume']].tail(3))
            
            df = add_indicators(df)
            
            if not df.empty:
                print("\nПример данных после расчета:")
                print(df[['close', 'RSI_14', 'OBV']].tail(3))
                df.to_csv(f"data_{exchange}_{symbol.replace('/', '')}.csv")
                print("Данные сохранены!")
            else:
                print("Нет данных после расчета индикаторов")
        else:
            print("Не удалось загрузить данные. Проверьте параметры:")
            print(f"- Корректность символа: {symbol}")
            print(f"- Доступность данных с {start_date}")
            
    except Exception as e:
        print(f"Ошибка выполнения: {e}")

Загрузка данных с binance...
Успешно загружено 876 свечей
Пример данных до индикаторов:
               close      volume
timestamp                       
2025-05-23  107271.5  279480.347
2025-05-24  107698.9  165158.758
2025-05-25  107544.2   84680.921
Ошибка расчета: module 'pandas_ta' has no attribute 'cdl_hammer'

Пример данных после расчета:
               close     RSI_14           OBV
timestamp                                    
2025-05-23  107271.5  63.256015  1.149632e+07
2025-05-24  107698.9  63.972558  1.166148e+07
2025-05-25  107544.2  63.489942  1.157680e+07
Данные сохранены!


In [13]:
df = pd.read_csv('data_binance_BTCUSDT.csv')

In [14]:
df

,timestamp,open,high,low,close,volume,SMA_20,EMA_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,OBV,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
0,2023-01-01,16537.5,16618.8,16488.0,16610.3,105502.965,NaN,NaN,NaN,NaN,NaN,NaN,1.055030e+05,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,16610.4,16799.0,16541.2,16666.0,215161.176,NaN,NaN,NaN,NaN,NaN,NaN,3.206641e+05,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,16665.9,16774.0,16600.3,16667.2,203070.205,NaN,NaN,NaN,NaN,NaN,NaN,5.237343e+05,NaN,NaN,NaN,NaN,NaN
3,2023-01-04,16667.3,16984.6,16645.7,16842.1,349747.838,NaN,NaN,NaN,NaN,NaN,NaN,8.734822e+05,NaN,NaN,NaN,NaN,NaN
4,2023-01-05,16842.2,16872.8,16740.4,16823.8,176369.347,NaN,NaN,NaN,NaN,NaN,NaN,6.971128e+05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,2025-05-21,106800.0,110830.0,106037.9,109609.5,413589.854,101960.735,96322.923676,75.587198,3952.277334,159.560023,3792.717310,1.152974e+07,93342.376802,101960.735,110579.093198,16.905249,0.943748
872,2025-05-22,109609.6,111959.5,109153.4,111662.7,246067.033,102702.310,96924.483532,78.069375,4228.969324,349.001611,3879.967713,1.177580e+07,93448.145359,102702.310,111956.474641,18.021337,0.984127
873,2025-05-23,111662.6,111770.4,106721.9,107271.5,279480.347,103275.720,97330.248884,63.256015,4047.262259,133.835637,3913.426622,1.149632e+07,94388.617055,103275.720,112162.822945,17.210440,0.724808
874,2025-05-24,107271.6,109440.2,106836.0,107698.9,165158.758,103949.155,97736.862653,63.972558,3892.871114,-16.444406,3909.315521,1.166148e+07,95904.581810,103949.155,111993.728190,15.477900,0.733061


In [43]:
import ccxt
import pandas as pd
import time

def fetch_ohlcv_data(exchange_name, symbol, timeframe='1h', limit=1000):
    exchange_class = getattr(ccxt, exchange_name)()
    exchange_class.load_markets()
    
    if symbol not in exchange_class.symbols:
        raise Exception(f"{symbol} not available on {exchange_name}")
    
    ohlcv = exchange_class.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Пример использования
df = fetch_ohlcv_data('binance', 'BTC/USDT', '1h', 500)
print(df.head())

            timestamp      open      high       low     close      volume
0 2025-05-04 19:00:00  95389.07  95640.98  95348.30  95584.07   198.19245
1 2025-05-04 20:00:00  95584.07  95735.00  95582.25  95719.99   158.45621
2 2025-05-04 21:00:00  95719.99  95758.08  95630.00  95675.99   158.95366
3 2025-05-04 22:00:00  95676.00  95690.48  94724.57  95015.82  1114.48671
4 2025-05-04 23:00:00  95015.82  95037.40  94151.38  94277.62  1308.44850


In [ ]:
# pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29421 sha256=596e1839902f01e016b2900c879fdd5a8f134f5a23012af139e73e769ccdfadb
  Stored in directory: c:\users\xndrf\appdata\local\pip\cache\wheels\5f\67\4f\8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\xndrf\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [44]:
df

,timestamp,open,high,low,close,volume
0,2025-05-04 19:00:00,95389.07,95640.98,95348.30,95584.07,198.19245
1,2025-05-04 20:00:00,95584.07,95735.00,95582.25,95719.99,158.45621
2,2025-05-04 21:00:00,95719.99,95758.08,95630.00,95675.99,158.95366
3,2025-05-04 22:00:00,95676.00,95690.48,94724.57,95015.82,1114.48671
4,2025-05-04 23:00:00,95015.82,95037.40,94151.38,94277.62,1308.44850
...,...,...,...,...,...,...
495,2025-05-25 10:00:00,107131.68,107390.89,106826.53,106946.40,780.43314
496,2025-05-25 11:00:00,106946.40,107272.73,106834.34,107260.00,587.12489
497,2025-05-25 12:00:00,107259.99,107625.00,107181.81,107530.57,799.16485
498,2025-05-25 13:00:00,107530.57,107743.34,107300.00,107521.75,446.62388


In [33]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
import ta

def add_technical_indicators(df):
    # Убедимся, что индикаторы не дублируются
    df = df.copy()

    # Трендовые индикаторы
    df['sma_20'] = ta.trend.SMAIndicator(close=df['close'], window=20).sma_indicator()
    df['ema_20'] = ta.trend.EMAIndicator(close=df['close'], window=20).ema_indicator()
    
    ichimoku = ta.trend.IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52)
    df['ichimoku_a'] = ichimoku.ichimoku_a()
    df['ichimoku_b'] = ichimoku.ichimoku_b()

    # Осцилляторы
    df['rsi'] = ta.momentum.RSIIndicator(close=df['close'], window=14).rsi()
    df['macd'] = ta.trend.MACD(close=df['close']).macd_diff()
    df['stochastic'] = ta.momentum.StochasticOscillator(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        window=14,
        smooth_window=3
    ).stoch()

    # # Объёмные индикаторы
    df['obv'] = ta.volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()
    df['vwap'] = ta.volume.VolumeWeightedAveragePrice(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        volume=df['volume'],
        window=14
    ).vwap()

    # Волатильность
    bollinger = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_bbm'] = bollinger.bollinger_mavg()
    df['bb_bbh'] = bollinger.bollinger_hband()
    df['bb_bbl'] = bollinger.bollinger_lband()

    df['atr'] = ta.volatility.AverageTrueRange(
        high=df['high'],
        low=df['low'],
        close=df['close'],
        window=14
    ).average_true_range()
    
    return df

# Применение к DataFrame df
df = add_technical_indicators(df)
print(df.tail())

              timestamp       open       high        low      close  \
495 2025-05-25 10:00:00  107131.68  107390.89  106826.53  106946.40   
496 2025-05-25 11:00:00  106946.40  107272.73  106834.34  107260.00   
497 2025-05-25 12:00:00  107259.99  107625.00  107181.81  107530.57   
498 2025-05-25 13:00:00  107530.57  107743.34  107300.00  107521.75   
499 2025-05-25 14:00:00  107521.74  107610.45  107432.61  107480.55   

        volume      sma_20         ema_20   ichimoku_a  ichimoku_b  ...  \
495  780.43314  108122.989  107919.178276  107803.1700  109045.295  ...   
496  587.12489  108039.504  107856.399393  107803.1700  109045.295  ...   
497  799.16485  107966.149  107825.368022  107803.1700  109045.295  ...   
498  446.62388  107896.235  107796.452020  107747.3650  109022.000  ...   
499  143.87019  107828.912  107766.366113  107691.7725  108900.740  ...   

           macd  stochastic      bb_bbm         bb_bbh         bb_bbl  \
495 -107.074037   11.018908  108122.989  109447.0

In [64]:
df

,timestamp,open,high,low,close,volume,sma_20,ema_20,ichimoku_a,ichimoku_b,...,macd,stochastic,bb_bbm,bb_bbh,bb_bbl,atr,hammer,shooting_star,engulfing,obv
0,2025-05-04 19:00:00,95389.07,95640.98,95348.30,95584.07,198.19245,NaN,NaN,NaN,95494.640,...,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0,198.192450
1,2025-05-04 20:00:00,95584.07,95735.00,95582.25,95719.99,158.45621,NaN,NaN,NaN,95541.650,...,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0,356.648660
2,2025-05-04 21:00:00,95719.99,95758.08,95630.00,95675.99,158.95366,NaN,NaN,NaN,95553.190,...,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0,197.695000
3,2025-05-04 22:00:00,95676.00,95690.48,94724.57,95015.82,1114.48671,NaN,NaN,NaN,95241.325,...,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0,-916.791710
4,2025-05-04 23:00:00,95015.82,95037.40,94151.38,94277.62,1308.44850,NaN,NaN,NaN,94954.730,...,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0,-2225.240210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2025-05-25 10:00:00,107131.68,107390.89,106826.53,106946.40,780.43314,108122.989,107919.178276,107803.1700,109045.295,...,-107.074037,11.018908,108122.989,109447.084414,106798.893586,546.784649,0,0,-100,20272.598305
496,2025-05-25 11:00:00,106946.40,107272.73,106834.34,107260.00,587.12489,108039.504,107856.399393,107803.1700,109045.295,...,-89.131481,26.124473,108039.504,109360.164122,106718.843878,539.042174,0,0,80,20859.723195
497,2025-05-25 12:00:00,107259.99,107625.00,107181.81,107530.57,799.16485,107966.149,107825.368022,107803.1700,109045.295,...,-53.194642,48.672682,107966.149,109227.420454,106704.877546,532.195590,0,0,0,21658.888045
498,2025-05-25 13:00:00,107530.57,107743.34,107300.00,107521.75,446.62388,107896.235,107796.452020,107747.3650,109022.000,...,-25.571979,51.450583,107896.235,109091.548396,106700.921604,525.848763,0,0,0,21212.264165


In [66]:
print(df[['high', 'low', 'close', 'volume']].dtypes)  # Проверьте типы данных


high      float64
low       float64
close     float64
volume    float64
dtype: object


In [67]:
df['high'].fillna(method='ffill', inplace=True)
df['low'].fillna(method='ffill', inplace=True)
df['close'].fillna(method='ffill', inplace=True)
df['volume'].fillna(0, inplace=True)  # Обратите внимание, что объем не должен быть NaN


C:\Users\xndrf\AppData\Local\Temp\ipykernel_23980\3051366376.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['high'].fillna(method='ffill', inplace=True)
C:\Users\xndrf\AppData\Local\Temp\ipykernel_23980\3051366376.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['low'].fillna(method='ffill', inplace=True)
C:\Users\xndrf\AppData\Local\Temp\ipykernel_23980\3051366376.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['close'].fillna(method='ffill', inplace=True)


In [70]:
df['high'].ffill(inplace=True)
df['low'].ffill(inplace=True)
df['close'].ffill(inplace=True)
df['volume'].fillna(0, inplace=True)  # Обратите внимание, что объем не должен быть NaN


In [71]:
import ta  # Убедитесь, что вы импортируете библиотеку ta

# Создание объекта VWAP
vwap_indicator = ta.volume.VolumeWeightedAveragePrice(
    high=df['high'],
    low=df['low'],
    close=df['close'],
    volume=df['volume'],
    window=14
)

# Получение значения VWAP
df['vwap'] = vwap_indicator.vwap()  # Убедитесь, что vwap() вызывается правильно


TypeError: 'Series' object is not callable

In [46]:
df

,timestamp,open,high,low,close,volume,sma_20,ema_20,ichimoku_a,ichimoku_b,rsi,macd,stochastic,bb_bbm,bb_bbh,bb_bbl,atr
0,2025-05-04 19:00:00,95389.07,95640.98,95348.30,95584.07,198.19245,NaN,NaN,NaN,95494.640,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2025-05-04 20:00:00,95584.07,95735.00,95582.25,95719.99,158.45621,NaN,NaN,NaN,95541.650,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,2025-05-04 21:00:00,95719.99,95758.08,95630.00,95675.99,158.95366,NaN,NaN,NaN,95553.190,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
3,2025-05-04 22:00:00,95676.00,95690.48,94724.57,95015.82,1114.48671,NaN,NaN,NaN,95241.325,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,2025-05-04 23:00:00,95015.82,95037.40,94151.38,94277.62,1308.44850,NaN,NaN,NaN,94954.730,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2025-05-25 10:00:00,107131.68,107390.89,106826.53,106946.40,780.43314,108122.989,107919.178276,107803.1700,109045.295,32.572616,-107.074037,11.018908,108122.989,109447.084414,106798.893586,546.784649
496,2025-05-25 11:00:00,106946.40,107272.73,106834.34,107260.00,587.12489,108039.504,107856.399393,107803.1700,109045.295,38.495348,-89.131481,26.124473,108039.504,109360.164122,106718.843878,539.042174
497,2025-05-25 12:00:00,107259.99,107625.00,107181.81,107530.57,799.16485,107966.149,107825.368022,107803.1700,109045.295,43.136321,-53.194642,48.672682,107966.149,109227.420454,106704.877546,532.195590
498,2025-05-25 13:00:00,107530.57,107743.34,107300.00,107521.75,446.62388,107896.235,107796.452020,107747.3650,109022.000,43.022356,-25.571979,51.450583,107896.235,109091.548396,106700.921604,525.848763


In [ ]:
# import ta

# def add_technical_indicators(df):
#     # Убедимся, что индикаторы не дублируются
#     df = df.copy()

#     # Трендовые индикаторы
#     df['sma_20'] = ta.trend.SMAIndicator(close=df['close'], window=20).sma_indicator()
#     df['ema_20'] = ta.trend.EMAIndicator(close=df['close'], window=20).ema_indicator()
    
#     ichimoku = ta.trend.IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52)
#     df['ichimoku_a'] = ichimoku.ichimoku_a()
#     df['ichimoku_b'] = ichimoku.ichimoku_b()

#     # Осцилляторы
#     df['rsi'] = ta.momentum.RSIIndicator(close=df['close'], window=14).rsi()
#     df['macd'] = ta.trend.MACD(close=df['close']).macd_diff()
#     df['stochastic'] = ta.momentum.StochasticOscillator(
#         high=df['high'],
#         low=df['low'],
#         close=df['close'],
#         window=14,
#         smooth_window=3
#     ).stoch()

#     # Объёмные индикаторы
#     df['obv'] = ta.volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()
#     df['vwap'] = ta.volume.VolumeWeightedAveragePrice(
#         high=df['high'],
#         low=df['low'],
#         close=df['close'],
#         volume=df['volume'],
#         window=14
#     ).vwap()

#     # Волатильность
#     bollinger = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
#     df['bb_bbm'] = bollinger.bollinger_mavg()
#     df['bb_bbh'] = bollinger.bollinger_hband()
#     df['bb_bbl'] = bollinger.bollinger_lband()

#     df['atr'] = ta.volatility.AverageTrueRange(
#         high=df['high'],
#         low=df['low'],
#         close=df['close'],
#         window=14
#     ).average_true_range()
    
#     return df

# # Применение к DataFrame df
# df = add_technical_indicators(df)
# print(df.tail())

TypeError: 'Series' object is not callable

In [54]:
import talib
print(talib.get_functions())


['HT_DCPERIOD', 'HT_DCPHASE', 'HT_PHASOR', 'HT_SINE', 'HT_TRENDMODE', 'ADD', 'DIV', 'MAX', 'MAXINDEX', 'MIN', 'MININDEX', 'MINMAX', 'MINMAXINDEX', 'MULT', 'SUB', 'SUM', 'ACOS', 'ASIN', 'ATAN', 'CEIL', 'COS', 'COSH', 'EXP', 'FLOOR', 'LN', 'LOG10', 'SIN', 'SINH', 'SQRT', 'TAN', 'TANH', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR',

In [55]:
import talib

def add_candlestick_patterns(df):
    open = df['open'].values
    high = df['high'].values
    low = df['low'].values
    close = df['close'].values

    df['hammer'] = talib.CDLHAMMER(open, high, low, close)
    df['shooting_star'] = talib.CDLSHOOTINGSTAR(open, high, low, close)
    df['engulfing'] = talib.CDLENGULFING(open, high, low, close)
    
    return df

# Assuming df is your DataFrame with OHLC data
df = add_candlestick_patterns(df)


In [56]:
df

,timestamp,open,high,low,close,volume,sma_20,ema_20,ichimoku_a,ichimoku_b,rsi,macd,stochastic,bb_bbm,bb_bbh,bb_bbl,atr,hammer,shooting_star,engulfing
0,2025-05-04 19:00:00,95389.07,95640.98,95348.30,95584.07,198.19245,NaN,NaN,NaN,95494.640,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
1,2025-05-04 20:00:00,95584.07,95735.00,95582.25,95719.99,158.45621,NaN,NaN,NaN,95541.650,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
2,2025-05-04 21:00:00,95719.99,95758.08,95630.00,95675.99,158.95366,NaN,NaN,NaN,95553.190,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
3,2025-05-04 22:00:00,95676.00,95690.48,94724.57,95015.82,1114.48671,NaN,NaN,NaN,95241.325,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
4,2025-05-04 23:00:00,95015.82,95037.40,94151.38,94277.62,1308.44850,NaN,NaN,NaN,94954.730,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2025-05-25 10:00:00,107131.68,107390.89,106826.53,106946.40,780.43314,108122.989,107919.178276,107803.1700,109045.295,32.572616,-107.074037,11.018908,108122.989,109447.084414,106798.893586,546.784649,0,0,-100
496,2025-05-25 11:00:00,106946.40,107272.73,106834.34,107260.00,587.12489,108039.504,107856.399393,107803.1700,109045.295,38.495348,-89.131481,26.124473,108039.504,109360.164122,106718.843878,539.042174,0,0,80
497,2025-05-25 12:00:00,107259.99,107625.00,107181.81,107530.57,799.16485,107966.149,107825.368022,107803.1700,109045.295,43.136321,-53.194642,48.672682,107966.149,109227.420454,106704.877546,532.195590,0,0,0
498,2025-05-25 13:00:00,107530.57,107743.34,107300.00,107521.75,446.62388,107896.235,107796.452020,107747.3650,109022.000,43.022356,-25.571979,51.450583,107896.235,109091.548396,106700.921604,525.848763,0,0,0


In [59]:
def prepare_ml_dataset(df):
    df = df.dropna()
    features = [
        'sma_20', 'ema_20', 'ichimoku_a', 'ichimoku_b',
        'rsi', 'macd', 'stochastic', #'obv', 'vwap',
        'bb_bbm', 'bb_bbh', 'bb_bbl', 'atr',
        'hammer', 'shooting_star', 'engulfing'
    ]
    target = df['close'].shift(-1) > df['close']  # бинарный таргет: 1 если цена вырастет
    return df[features], target.astype(int)

X, y = prepare_ml_dataset(df)

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      0.52      0.54        50
           1       0.50      0.55      0.52        44

    accuracy                           0.53        94
   macro avg       0.53      0.53      0.53        94
weighted avg       0.53      0.53      0.53        94



In [61]:
df

,timestamp,open,high,low,close,volume,sma_20,ema_20,ichimoku_a,ichimoku_b,rsi,macd,stochastic,bb_bbm,bb_bbh,bb_bbl,atr,hammer,shooting_star,engulfing
0,2025-05-04 19:00:00,95389.07,95640.98,95348.30,95584.07,198.19245,NaN,NaN,NaN,95494.640,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
1,2025-05-04 20:00:00,95584.07,95735.00,95582.25,95719.99,158.45621,NaN,NaN,NaN,95541.650,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
2,2025-05-04 21:00:00,95719.99,95758.08,95630.00,95675.99,158.95366,NaN,NaN,NaN,95553.190,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
3,2025-05-04 22:00:00,95676.00,95690.48,94724.57,95015.82,1114.48671,NaN,NaN,NaN,95241.325,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
4,2025-05-04 23:00:00,95015.82,95037.40,94151.38,94277.62,1308.44850,NaN,NaN,NaN,94954.730,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2025-05-25 10:00:00,107131.68,107390.89,106826.53,106946.40,780.43314,108122.989,107919.178276,107803.1700,109045.295,32.572616,-107.074037,11.018908,108122.989,109447.084414,106798.893586,546.784649,0,0,-100
496,2025-05-25 11:00:00,106946.40,107272.73,106834.34,107260.00,587.12489,108039.504,107856.399393,107803.1700,109045.295,38.495348,-89.131481,26.124473,108039.504,109360.164122,106718.843878,539.042174,0,0,80
497,2025-05-25 12:00:00,107259.99,107625.00,107181.81,107530.57,799.16485,107966.149,107825.368022,107803.1700,109045.295,43.136321,-53.194642,48.672682,107966.149,109227.420454,106704.877546,532.195590,0,0,0
498,2025-05-25 13:00:00,107530.57,107743.34,107300.00,107521.75,446.62388,107896.235,107796.452020,107747.3650,109022.000,43.022356,-25.571979,51.450583,107896.235,109091.548396,106700.921604,525.848763,0,0,0


6. Отправка сигналов на Bybit через Webhook

In [ ]:
# import requests
# from datetime import datetime

# def send_webhook(symbol, side, qty, price, strategy_id="test123"):
#     url = 'https://api.bybit.com/custom_webhook_endpoint'
#     data = {
#         "symbol": symbol,
#         "side": side,
#         "qty": str(qty),
#         "price": str(price),
#         "trigger_time": datetime.utcnow().isoformat(),
#         "max_lag": "20",
#         "strategy_id": strategy_id
#     }
#     response = requests.post(url, json=data)
#     print(response.status_code, response.text)

# # Пример сигнала
# send_webhook("BTC/USDT", "buy", 1, df['close'].iloc[-1])